In [3]:
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np

#from DeformableConv2d import ConvOffset2d   # My implementation
from torch_deform_conv.layers import ConvOffset2D as ConvOffset2d  # git me
#from torch_deform_conv.deform_conv2 import DeformConv2d as ConvOffset2d  # git Lin lv
#import DeformableResNet
print(torch.__version__)

tmp = torch.randn(10,512,7,7)
print(tmp.mean(3).mean(2).shape)

1.1.0
torch.Size([10, 512])


In [7]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [12]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        #self.deformConv = ConvOffset2d(3)  this is where we adopt deformable conv
        self.conv1 = nn.Conv2d(3,6,5)
        self.pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(6,16,5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
    def forward(self, x):
        #x = self.deformConv(x)
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.view(-1, 16*5*5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
net = Net()
print(net)

# Define Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr = 0.001, momentum=0.9 )

Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


# Train the network

In [13]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        x, labels = data
        output = net(x)
        optimizer.zero_grad()
    
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss
        
        if i % 2000 == 1999:
            print('Epoch:'+ str(epoch+1) + ', '+ str(i+1) + ', loss:'
                 + str(running_loss/2000) )
        
        #print('Epoch:'+ str(epoch+1) + ', '+ str(i+1) + ', loss:'
                 #+ str(running_loss/2000) )
print('Complete training!')
        

Epoch:1, 2000, loss:tensor(2.2779, grad_fn=<DivBackward0>)
Epoch:1, 4000, loss:tensor(4.2606, grad_fn=<DivBackward0>)
Epoch:1, 6000, loss:tensor(5.9910, grad_fn=<DivBackward0>)
Epoch:1, 8000, loss:tensor(7.6334, grad_fn=<DivBackward0>)
Epoch:1, 10000, loss:tensor(9.1729, grad_fn=<DivBackward0>)
Epoch:1, 12000, loss:tensor(10.6665, grad_fn=<DivBackward0>)
Epoch:2, 2000, loss:tensor(1.4069, grad_fn=<DivBackward0>)
Epoch:2, 4000, loss:tensor(2.7747, grad_fn=<DivBackward0>)
Epoch:2, 6000, loss:tensor(4.1159, grad_fn=<DivBackward0>)
Epoch:2, 8000, loss:tensor(5.4213, grad_fn=<DivBackward0>)
Epoch:2, 10000, loss:tensor(6.7116, grad_fn=<DivBackward0>)
Epoch:2, 12000, loss:tensor(7.9854, grad_fn=<DivBackward0>)
Complete training!


# Test the classifier

In [14]:
import matplotlib.pyplot as plt
import numpy as np

# functions to show an image


def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


# get some random training images
dataiter = iter(trainloader)
images, labels = dataiter.next()

# show images
#imshow(torchvision.utils.make_grid(images))
# print labels
#print(' '.join('%5s' % classes[labels[j]] for j in range(4)))

dataiter = iter(testloader)
images, labels = dataiter.next()

# print images
imshow(torchvision.utils.make_grid(images))
print('GroundTruth: ', ' '.join('%5s' % classes[labels[j]] for j in range(4)))

# START  TEST
outputs = net(images)
_, predicted = torch.max(outputs, 1)

print('Predicted: ', ' '.join('%5s' % classes[predicted[j]]
                              for j in range(4)))

<Figure size 640x480 with 1 Axes>

GroundTruth:    cat  ship  ship plane
Predicted:    cat   car   car  ship


In [15]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))


Accuracy of the network on the 10000 test images: 54 %


In [16]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

Accuracy of plane : 57 %
Accuracy of   car : 68 %
Accuracy of  bird : 29 %
Accuracy of   cat : 30 %
Accuracy of  deer : 37 %
Accuracy of   dog : 45 %
Accuracy of  frog : 74 %
Accuracy of horse : 67 %
Accuracy of  ship : 68 %
Accuracy of truck : 70 %


# Using built-in torchvision.models.resnet

In [125]:
#print(resnet1)